## QA RAG

In [3]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from pprint import pprint

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("http://www.infopub.co.kr/new/include/detail.asp?sku=05000274",),
)
docs = loader.load()

print(len(docs))
print(docs[0])
pprint(docs[0].metadata)

USER_AGENT environment variable not set, consider setting it to identify your requests.


1
page_content='\n\n\n\n\n[정보문화사] IT 도서의 새로운 패러다임\n\n\n\n\nCJ스타일베너[좌슬라이드]\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\xa0\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\r\n파이썬 머신러닝 판다스 데이터 분석 개정판\r\n\n\n\n저자:\n\n오승환\n\n\n\n역자:\n\n\n\n\n구분:\n\r\n\t\r\n\t\r\n       국내서\r\n    \r\n\t\r\n\t\n\n\n발행일:\n2024년 6월 25일\n\n\n정가:\n30,000원\n\n\n페이지:\n584 페이지\n\n\nISBN:\n978-89-5674-980-8\n\n\n출판사:\n\n\r\n\t\t\t\t정보문화사\r\n\r\n\t\n\n\n판형:\n187×235\n\n\n\n난이도:\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n책소개\n저자소개\n미리보기\n오탈자\n보도자료실\n샘플자료 다운로드\n\n\n\n데이터 과학자가 되기 위한 첫걸음!어려운 이론은 최소화하고, 예제 코드를 따라 하며 자연스럽게 사용법에 익숙해지도록 안내하는 데이터 분석 입문서다. 데이터 분석을 처음 배우는 입문자의 입장에서 고급 이론과 데이터 분석 도구를 함께 배우는 것의 어려움을 아는 저자가, 데이터 분석에 필요한 필수 라이브러리를 소개하고 설치부터 예제 코드까지 따라 할 수 있게 구성했다. 개념 이해를 돕기 위해 다이어그램 등 풍부한 도식화도 적극 활용했다. 1판에서 큰 인기를 얻어 준비된 이번 개정판에서는 저자가 실무에서 쌓은 경험을 자연스럽게 녹이기 위해 노력했다. 여러 기업과 대학에서 계속

In [4]:
# HugoingFace Embeddings를 다운로드
from langchain.embeddings import HuggingFaceEmbeddings

embeddings_model = HuggingFaceEmbeddings(
    model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS",
)

# HugoingFace Embedding 모델의 Tokenizer를 사용하여 토큰화
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

/Users/wiggler/Library/Caches/pypoetry/virtualenvs/langfuse-practice-ygCFMd-0-py3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/Users/wiggler/Library/Caches/pypoetry/virtualenvs/langfuse-practice-ygCFMd-0-py3.11/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [5]:
# Token 수를 기준으ㄹ 문서를 청크 단위로 분할
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer = tokenizer,
    chunk_size = 120,
    chunk_overlap  = 10,
)

split_docs = text_splitter.split_documents(docs)
print(len(split_docs))
print(split_docs[0])

36
page_content='[정보문화사] IT 도서의 새로운 패러다임\n\n\n\n\nCJ스타일베너[좌슬라이드]' metadata={'source': 'http://www.infopub.co.kr/new/include/detail.asp?sku=05000274', 'title': '[정보문화사] IT 도서의 새로운 패러다임', 'language': 'No language found.'}


In [7]:
vectorstore = Chroma.from_documents(documents=split_docs, 
                                    embedding=embeddings_model)

retriever = vectorstore.as_retriever()

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# Chat Model
llm = ChatOllama(model="qwen2")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# RAG Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Chain 실행
response = rag_chain.invoke("이 책의 특징을 3가지 요점으로 설명해주세요.")
print(response)

1. **시작부터 전반적인 내용까지 자세히 가이드**: 이 책은 Pandas 2.0 버전의 새로운 기능과 변경된 부분들을 포함하여 다양한 기법과 자료구조, 그리고 기본 문법을 수강하는 비전공자와 입문자를 위한 구성되어 있습니다. 책에는 개념 설명을 돕기 위해 다이어그램 등의 풍부한 도식화를 활용하고 있습니다.

2. **실습 중심의 학습 방법**: 이 책은 실습 중심으로 진행하여 지루하고 복잡한 이론을 최소화하는 것이 특징입니다. 이렇게 하여 독자들은 이론을 실제로 경험하면서 실력을 향상시킬 수 있습니다. 

3. **공식적인 설치부터 시작과 코드 예제까지 가이드**: 이 책은 Pandas의 설치 방법부터 시작해서 다양한 코드 예제를 통해 배우게 되며, 이런 구성 방식으로 독자들은 한 권의 책으로 정확하고 빠르게 실력을 향상시킬 수 있습니다.


## Test Data 만들기 - AutoRAG 활용

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from autorag.data.corpus import langchain_documents_to_parquet
corpus_df = langchain_documents_to_parquet(split_docs, 'corpus_data/pandas_book.parquet')
corpus_df.head()

,doc_id,contents,metadata
0,6044ced6-79fc-402f-8908-e5dd135cce70,[정보문화사] IT 도서의 새로운 패러다임\n\n\n\n\nCJ스타일베너[좌슬라이드],{'filename': 'http://www.infopub.co.kr/new/inc...
1,006209f3-3283-4b8d-927a-0672a1409b69,파이썬 머신러닝 판다스 데이터 분석 개정판\r\n\n\n\n저자:,{'filename': 'http://www.infopub.co.kr/new/inc...
2,8fa203ba-c2e9-4d52-baca-22735b4959b9,저자:\n\n오승환\n\n\n\n역자:\n\n\n\n\n구분:\n\r\n\t\r\n...,{'filename': 'http://www.infopub.co.kr/new/inc...
3,801572ee-935c-4d94-a2c8-84c0b5c3459d,난이도:,{'filename': 'http://www.infopub.co.kr/new/inc...
4,2208cb11-03d1-4de5-8db2-e014f6c5a5a8,책소개\n저자소개\n미리보기\n오탈자\n보도자료실\n샘플자료 다운로드,{'filename': 'http://www.infopub.co.kr/new/inc...


In [ ]:
import pandas as pd

from autorag.data.qacreation import generate_qa_llama_index, make_single_content_qa
from llama_index.llms.ollama import Ollama


prompt = """
Generate question and answer pairs for the given passage.

Passage:
{{text}}

Number of questions to generate: {{num_questions}}

Example:
[Q]: 이 책의 글쓴이는 누구인가요?
[A]: 저자의 이름은 홍길동입니다.

Result:
"""

corpus_df = pd.read_parquet('corpus_data/pandas_book.parquet')

llm = Ollama(model='gemma2', temperature=1.0)
qa_df = make_single_content_qa(corpus_df, content_size=36, qa_creation_func=generate_qa_llama_index,
                            llm=llm, prompt=prompt, question_num_per_content=1)

qa_df.head()

,qid,retrieval_gt,query,generation_gt
0,4c92e4e1-4fa7-44a9-a467-5e9b587ec5fe,[[70bf58d5-f8ca-4e66-af72-f64d7f12f40b]],이 자료의 크기는 얼마입니까?,[9267KB입니다.]
1,487c21eb-ea11-4b7d-b439-624635f2dad2,[[8db88b36-5acc-4663-b804-f2a0f1b1fe32]],이 책에서 어떤 새로운 내용이 추가되었나요?,[등의 기법과 사례 등을 더해 150쪽 이상의 분량이 더해졌다. 특히 시계열 데이터...
2,7bfc7453-0c88-4ea8-9357-00d2b6602ef5,[[5a7f7a80-9fc5-4f25-ab04-6b4b21d300cb]],챗GPT 프롬프트 120% 질문 기술에 대한 보도자료는 언제 발표되었습니다?,[2024년 4월 24일.]
3,119a1d83-aee7-4b04-abe9-0c8ea3c38cfe,[[a7f166ac-b497-43b3-bc96-dc080ad5092b]],2023년 7월 19일에 발행된 자료는 무엇인가요?,[파이썬과 엑셀로 시작하는 딥러닝입니다.]
4,0358ffc9-6334-4dda-ab92-26e7d21a60b3,[[e99c381f-326e-4dd9-a130-6d0b0d4744a6]],시간에 따른 데이터 변화를 분석하기 위해 어떤 개선 사항이 있었나요?,[대폭 추가되어 시간에 따른 데이터 변화를 효과적으로 분석할 수 있도록 했습니다.]


In [ ]:
qa_df.to_excel('corpus_data/pandas_book_qa.xlsx', index=False)

In [9]:
# 직접 수정한 QA 데이터를 불러옴
qa_data = pd.read_excel('corpus_data/pandas_book_qa_final.xlsx')
qa_data.head()

,qid,retrieval_gt,query,generation_gt
0,7488c947-8e27-4747-b342-074edbcdde31,[['e63e3b0c-97ed-4f31-b64d-bb60525969be']],이 책의 개정판은 어떤 점이 추가되었나요?,['최신 버전인 Pandas 2.0 버전에서 추가되거나 변경된 내용을 포함하고 있습...
1,64a6335b-b871-44a7-8b36-170e151983a1,[['e73993c7-0a14-4702-8a21-0582076e9c10']],데이터 분석의 기초를 더욱 탄탄하게 다지기 위해 어떤 기법을 추가했나요?,"['데이터 정제, 변환, 통합 등 다양한 데이터 전처리 기법을 추가했습니다.']"
2,1152af6c-e9df-42f0-901d-4f8f2ff471f5,[['2208cb11-03d1-4de5-8db2-e014f6c5a5a8']],이 웹 페이지에서 샘플 자료를 다운로드할 수 있나요?,"['네, 샘플자료 다운로드 버튼을 통해 샘플 자료를 다운로드할 수 있습니다.']"
3,bbf40b27-bc8f-4848-977d-098f276437f7,[['aad2ab82-eed4-4e99-9dc0-06a4d3470142']],데이터 분석에서 시계열 데이터를 처리하는 방법은 무엇인가요?,['시계열 데이터 분석에서 중요한 부분이기 때문에 다른 자료형을 시계열 객체로 변환...
4,ea6d1f18-67ad-4390-95df-9de430210af1,[['5387af25-ec8b-449e-a4da-f8685c27178a']],이 책이 데이터 과학자를 되기 위한 첫걸음이라고 하는 이유는 무엇인가요?,['이 책은 어려운 이론을 최소화하고 예제 코드를 따라 하며 사용법에 익숙해지도록 ...


In [ ]:
qa_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   qid            11 non-null     object
 1   retrieval_gt   11 non-null     object
 2   query          11 non-null     object
 3   generation_gt  11 non-null     object
dtypes: object(4)
memory usage: 484.0+ bytes


In [ ]:
type(qa_data['retrieval_gt'][0])

str

In [ ]:
type(qa_data['generation_gt'][0])

str

In [10]:
import ast
for col in ['retrieval_gt', 'generation_gt']:
    qa_data[col] = qa_data[col].apply(ast.literal_eval)

qa_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   qid            11 non-null     object
 1   retrieval_gt   11 non-null     object
 2   query          11 non-null     object
 3   generation_gt  11 non-null     object
dtypes: object(4)
memory usage: 484.0+ bytes


In [11]:
type(qa_data['retrieval_gt'][0])

list

In [12]:
type(qa_data['generation_gt'][0])

list

## RAGAS - RAG Evaluation

In [ ]:
# Chain 테스트
response = rag_chain.invoke("이 책의 출판사는 어디인가요?")
print(response)

이 책의 출판사는 '정보문화사'입니다.


In [13]:
qa_data = qa_data[['query', 'generation_gt']]
qa_data.columns = ['question', 'ground_truth']
qa_data.head()

,question,ground_truth
0,이 책의 개정판은 어떤 점이 추가되었나요?,[최신 버전인 Pandas 2.0 버전에서 추가되거나 변경된 내용을 포함하고 있습니다.]
1,데이터 분석의 기초를 더욱 탄탄하게 다지기 위해 어떤 기법을 추가했나요?,"[데이터 정제, 변환, 통합 등 다양한 데이터 전처리 기법을 추가했습니다.]"
2,이 웹 페이지에서 샘플 자료를 다운로드할 수 있나요?,"[네, 샘플자료 다운로드 버튼을 통해 샘플 자료를 다운로드할 수 있습니다.]"
3,데이터 분석에서 시계열 데이터를 처리하는 방법은 무엇인가요?,[시계열 데이터 분석에서 중요한 부분이기 때문에 다른 자료형을 시계열 객체로 변환하...
4,이 책이 데이터 과학자를 되기 위한 첫걸음이라고 하는 이유는 무엇인가요?,[이 책은 어려운 이론을 최소화하고 예제 코드를 따라 하며 사용법에 익숙해지도록 안...


In [185]:
qa_data['ground_truth'] = qa_data['ground_truth'].apply(lambda x: x[0] )
qa_data['answer'] = qa_data['question'].apply(lambda x: rag_chain.invoke(x) )
qa_data['contexts'] = qa_data['question'].apply(lambda x: [d.page_content for d in retriever.get_relevant_documents(x)] )
qa_data.head()

/var/folders/vp/t7xb2kg161q5m2ylkq9jn7k00000gn/T/ipykernel_46596/1755535353.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qa_data['ground_truth'] = qa_data['ground_truth'].apply(lambda x: x[0] )
/var/folders/vp/t7xb2kg161q5m2ylkq9jn7k00000gn/T/ipykernel_46596/1755535353.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qa_data['answer'] = qa_data['question'].apply(lambda x: rag_chain.invoke(x) )


,question,ground_truth,answer,contexts
0,이 책의 개정판은 어떤 점이 추가되었나요?,최신 버전인 Pandas 2.0 버전에서 추가되거나 변경된 내용을 포함하고 있습니다.,"이 책의 개정판에는 Pandas 2.0 버전에서 추가되거나 변경된 내용, 시계열 데...",[비전공자와 입문자의 마음을 사로잡은 강의가 개정판으로 돌아왔다. 최신 버전인 Pa...
1,데이터 분석의 기초를 더욱 탄탄하게 다지기 위해 어떤 기법을 추가했나요?,"데이터 정제, 변환, 통합 등 다양한 데이터 전처리 기법을 추가했습니다.","데이터 분석의 기초를 더욱 탄탄하게 다지기 위해 추가된 기법은 ""데이터 정제, 변환...","[다양화했다. 데이터 정제, 변환, 통합 등 다양한 데이터 전처리 기법을 추가하여 ..."
2,이 웹 페이지에서 샘플 자료를 다운로드할 수 있나요?,"네, 샘플자료 다운로드 버튼을 통해 샘플 자료를 다운로드할 수 있습니다.","문献提供的描述中并没有明确提到是否可以下载样例资料。但考虑到页面提及了""출판사 서평예제 중...",[점이나 책과 관련된 요청 사항을 전달할 수 있다. 출판사 서평예제 중심으로 실무용...
3,데이터 분석에서 시계열 데이터를 처리하는 방법은 무엇인가요?,시계열 데이터 분석에서 중요한 부분이기 때문에 다른 자료형을 시계열 객체로 변환하는...,데이터 분석에서 시계열 데이터를 처리하는 방법에는 여러 가지가 있습니다.\n\n1....,[데이터에 특히 많이 쓰인다. 데이터 분석에서 중요한 부분이기 때문에 다른 자료형을...
4,이 책이 데이터 과학자를 되기 위한 첫걸음이라고 하는 이유는 무엇인가요?,이 책은 어려운 이론을 최소화하고 예제 코드를 따라 하며 사용법에 익숙해지도록 안내...,이 책이 '데이터 과학자를 되기 위한 첫 걸음'이라고 할 수 있는 이유는 다음과 같...,"[데이터 과학자가 되기 위한 첫걸음!어려운 이론은 최소화하고, 예제 코드를 따라 하..."


In [187]:
from datasets import Dataset 

dataset = Dataset.from_pandas(qa_data)

dataset[0]

{'question': '이 책의 개정판은 어떤 점이 추가되었나요?',
 'ground_truth': '최신 버전인 Pandas 2.0 버전에서 추가되거나 변경된 내용을 포함하고 있습니다.',
 'answer': '이 책의 개정판에는 Pandas 2.0 버전에서 추가되거나 변경된 내용, 시계열 데이터 처리 등의 기법과 다양한 데이터 시각화, 데이터 전처리 기법 등이 추가되었습니다.',
 'contexts': ['비전공자와 입문자의 마음을 사로잡은 강의가 개정판으로 돌아왔다. 최신 버전인 Pandas 2.0 버전에서 추가되거나 변경된 내용을 포함하여 시계열 데이터 처리 등의 기법과',
  '만큼 더욱 자세한 설명과 다양한 데이터 시각화, 데이터 전처리 기법을 담았으며, Pandas 2.0 버전에서 추가되거나 변경된 내용을 풍부하게 담아 데이터 분석에',
  '큰 인기를 얻어 준비된 이번 개정판에서는 저자가 실무에서 쌓은 경험을 자연스럽게 녹이기 위해 노력했다. 여러 기업과 대학에서 계속 강의를 해오고 있는 만큼 더욱',
  '소개하고 설치부터 예제 코드까지 따라 할 수 있게 구성했다. 개념 이해를 돕기 위해 다이어그램 등 풍부한 도식화도 적극 활용했다. 1판에서 큰 인기를']}

In [188]:
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

In [189]:
from ragas import evaluate
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings.ollama import OllamaEmbeddings

langchain_llm =  ChatOllama(model="gemma2")
langchain_embeddings = OllamaEmbeddings(model="gemma2")

result = evaluate(
    dataset,
    metrics = [
        faithfulness,
        answer_relevancy,
        context_recall,
        context_precision,
    ],
    llm=langchain_llm, 
    embeddings=langchain_embeddings,
    raise_exceptions=False,
)

result

Evaluating:   0%|          | 0/44 [00:00<?, ?it/s]

[06/29/24 05:27:25] WARNING  [output_parser.py:71] >> Failed to parse output. Returning None.   ]8;id=346523;file:///Users/wiggler/Library/Caches/pypoetry/virtualenvs/langfuse-practice-ygCFMd-0-py3.11/lib/python3.11/site-packages/ragas/llms/output_parser.py\output_parser.py]8;;\:]8;id=495101;file:///Users/wiggler/Library/Caches/pypoetry/virtualenvs/langfuse-practice-ygCFMd-0-py3.11/lib/python3.11/site-packages/ragas/llms/output_parser.py#71\71]8;;\

{'faithfulness': 0.9076, 'answer_relevancy': 0.3192, 'context_recall': 0.9000, 'context_precision': 0.9672}

## Gemma2 모델을 사용한 RAG Chain

In [190]:
# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# Chat Model
llm = ChatOllama(model="gemma2")

rag_chain2 = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

qa_data2 = qa_data.copy()
qa_data2['answer'] = qa_data2['question'].apply(lambda x: rag_chain2.invoke(x) )
qa_data2.head()

,question,ground_truth,answer,contexts
0,이 책의 개정판은 어떤 점이 추가되었나요?,최신 버전인 Pandas 2.0 버전에서 추가되거나 변경된 내용을 포함하고 있습니다.,이 책의 개정판에서는 다음과 같은 점이 추가되었습니다.\n\n* Pandas 2.0...,[비전공자와 입문자의 마음을 사로잡은 강의가 개정판으로 돌아왔다. 최신 버전인 Pa...
1,데이터 분석의 기초를 더욱 탄탄하게 다지기 위해 어떤 기법을 추가했나요?,"데이터 정제, 변환, 통합 등 다양한 데이터 전처리 기법을 추가했습니다.","데이터 정제, 변환, 통합 등 다양한 데이터 전처리 기법을 추가했습니다. \n","[다양화했다. 데이터 정제, 변환, 통합 등 다양한 데이터 전처리 기법을 추가하여 ..."
2,이 웹 페이지에서 샘플 자료를 다운로드할 수 있나요?,"네, 샘플자료 다운로드 버튼을 통해 샘플 자료를 다운로드할 수 있습니다.",The provided text does not mention anything ab...,[점이나 책과 관련된 요청 사항을 전달할 수 있다. 출판사 서평예제 중심으로 실무용...
3,데이터 분석에서 시계열 데이터를 처리하는 방법은 무엇인가요?,시계열 데이터 분석에서 중요한 부분이기 때문에 다른 자료형을 시계열 객체로 변환하는...,"시계열 데이터 처리 기법, 시계열 데이터 변환, 시계열 데이터 분석 등 다양한 방법...",[데이터에 특히 많이 쓰인다. 데이터 분석에서 중요한 부분이기 때문에 다른 자료형을...
4,이 책이 데이터 과학자를 되기 위한 첫걸음이라고 하는 이유는 무엇인가요?,이 책은 어려운 이론을 최소화하고 예제 코드를 따라 하며 사용법에 익숙해지도록 안내...,이 책이 데이터 과학자를 되기 위한 첫걸음이라고 하는 이유는 어려운 이론을 최소화하...,"[데이터 과학자가 되기 위한 첫걸음!어려운 이론은 최소화하고, 예제 코드를 따라 하..."


In [191]:
dataset2 = Dataset.from_pandas(qa_data2)

result2 = evaluate(
    dataset2,
    metrics = [
        faithfulness,
        answer_relevancy,
        context_recall,
        context_precision,
    ],
    llm=langchain_llm, 
    embeddings=langchain_embeddings,
    raise_exceptions=False,
)

result2

Evaluating:   0%|          | 0/44 [00:00<?, ?it/s]

[06/29/24 05:40:24] WARNING  [output_parser.py:71] >> Failed to parse output. Returning None.   ]8;id=567075;file:///Users/wiggler/Library/Caches/pypoetry/virtualenvs/langfuse-practice-ygCFMd-0-py3.11/lib/python3.11/site-packages/ragas/llms/output_parser.py\output_parser.py]8;;\:]8;id=816044;file:///Users/wiggler/Library/Caches/pypoetry/virtualenvs/langfuse-practice-ygCFMd-0-py3.11/lib/python3.11/site-packages/ragas/llms/output_parser.py#71\71]8;;\

[06/29/24 05:43:03] WARNING  [output_parser.py:71] >> Failed to parse output. Returning None.   ]8;id=821002;file:///Users/wiggler/Library/Caches/pypoetry/virtualenvs/langfuse-practice-ygCFMd-0-py3.11/lib/python3.11/site-packages/ragas/llms/output_parser.py\output_parser.py]8;;\:]8;id=28536;file:///Users/wiggler/Library/Caches/pypoetry/virtualenvs/langfuse-practice-ygCFMd-0-py3.11/lib/python3.11/site-packages/ragas/llms/output_parser.py#71\71]8;;\

{'faithfulness': 0.9000, 'answer_relevancy': 0.3450, 'context_recall': 0.9000, 'context_precision': 0.9672}

## 모델 성능 비교

In [192]:
df1 = pd.DataFrame(list(result.items()), columns=['Metric', 'Qwen2'])
df2 = pd.DataFrame(list(result2.items()), columns=['Metric', 'Gemma2'])

df_result = pd.merge(df1, df2, on='Metric')
df_result

,Metric,Qwen2,Gemma2
0,faithfulness,0.907576,0.900000
1,answer_relevancy,0.319209,0.345019
2,context_recall,0.900000,0.900000
3,context_precision,0.967172,0.967172
